LOAD LIBRARIES

In [1]:
import pandas as pd
from datetime import datetime, timedelta

REQUESTS

In [2]:
import os
import glob
import pandas as pd
from datetime import datetime

# ───── CONSTANTS ─────
FILES_DIR      = "Files"
RC_FILE_NAME   = "Trade Requests - Last 3 RCsh.xlsx"
LIQ_FILE_NAME  = "Trade Requests - Last 3 AcctLiq Custom.xlsx"
ACCOUNT_ID_CSV = os.path.join(FILES_DIR, "Constant", "AccountID.csv")

# Column names
RC_ACCT_COL      = 'Related Process: Account Number'
RC_GROSS_COL     = 'Related Process: Gross Amount Requested'
RC_MINSET_COL    = 'Related Process: Amount to be set aside'
PSC_MODEL_COL    = 'New Model Trading'

LIQ_ACCT_COL     = 'Related Process: Account Number'
TRADE_TYPE_COL   = 'Related Process: Trade Request Type(s)'
IS_CLOSING_COL   = 'Related Process: Is the Account closing?'

today_str = datetime.today().strftime('%m-%d-%Y')

# ───── Load Account-ID lookup ─────
acct_id_df = pd.read_csv(ACCOUNT_ID_CSV, dtype=str)
acct_id_df.rename(columns={'Account Number': 'ACCOUNT NUMBER'}, inplace=True)

# ───── 1) Process Raise-Cash file ─────
rc_path = os.path.join(FILES_DIR, RC_FILE_NAME)
if not os.path.exists(rc_path):
    raise FileNotFoundError(f"Cannot find {rc_path}")
df_rc = pd.read_excel(rc_path)

# A) Raise-Cash & Cash-Minimum
cash_rows = []
for _, r in df_rc.iterrows():
    acct = r.get(RC_ACCT_COL)
    if pd.isna(acct):
        continue

    gross = r.get(RC_GROSS_COL)
    if pd.notna(gross):
        cash_rows.append({
            'Account ID': None,
            'Account Number': str(acct),
            'Set Aside Type ($ or %)': '$',
            'Set Aside Amount': gross,
            'Set Aside Minimum': gross,
            'Set Aside Maximum': None,
            'Expiration Type': 'Transaction',
            'Expiration Date': None,
            'Transaction Type': 'Distribution/Merge Out',
            'Transaction Tolerance Band (%)': 40,
            'Percent Calculation Type': None,
            'Description': 'Raise Cash',
            'Start Date': today_str
        })

    minimum = r.get(RC_MINSET_COL)
    if pd.notna(minimum):
        cash_rows.append({
            'Account ID': None,
            'Account Number': str(acct),
            'Set Aside Type ($ or %)': '$',
            'Set Aside Amount': minimum,
            'Set Aside Minimum': minimum * 3,
            'Set Aside Maximum': None,
            'Expiration Type': None,
            'Expiration Date': None,
            'Transaction Type': None,
            'Transaction Tolerance Band (%)': None,
            'Percent Calculation Type': None,
            'Description': 'Cash Minimum',
            'Start Date': today_str
        })

# Export Raise Cash
out_cash_df = pd.DataFrame(cash_rows)
out_cash_df.to_excel("Raise_Cash_Requests.xlsx", index=False)
print(f"Generated {len(out_cash_df)} Raise Cash rows → Raise_Cash_Requests.xlsx")

# B) PSC / New-Model from same file
df_psc = df_rc[df_rc[PSC_MODEL_COL].notna()].copy()
df_psc[RC_ACCT_COL] = df_psc[RC_ACCT_COL].astype(str)
df_psc = df_psc.merge(
    acct_id_df[['ACCOUNT NUMBER','Account ID']],
    left_on=RC_ACCT_COL, right_on='ACCOUNT NUMBER', how='left'
)

psc_rows = []
for _, r in df_psc.iterrows():
    psc_rows.append({
        'Account Change History ID': 0,
        'Account ID': r['Account ID'],
        'Change Type': 'Aggregated Model',
        'Change Date': today_str,
        'Next': r[PSC_MODEL_COL],
        'Remove Account Change': None
    })

# ───── 2) Process Liquidation file ─────
liq_path = os.path.join(FILES_DIR, LIQ_FILE_NAME)
if not os.path.exists(liq_path):
    raise FileNotFoundError(f"Cannot find {liq_path}")
df_liq = pd.read_excel(liq_path)

df_liq = df_liq[df_liq[TRADE_TYPE_COL]=='Account Liquidation'].copy()
df_liq[LIQ_ACCT_COL] = df_liq[LIQ_ACCT_COL].astype(str)
df_liq = df_liq.merge(
    acct_id_df[['ACCOUNT NUMBER','Account ID']],
    left_on=LIQ_ACCT_COL, right_on='ACCOUNT NUMBER', how='left'
)

liq_rows = []
for _, r in df_liq.iterrows():
    next_flag = (
        'Liquidated Closed'
        if str(r[IS_CLOSING_COL]).strip().lower()=='yes'
        else 'Liquidated Open'
    )
    liq_rows.append({
        'Account Change History ID': 0,
        'Account ID': r['Account ID'],
        'Change Type': 'Account Liquidation',
        'Change Date': today_str,
        'Next': next_flag,
        'Remove Account Change': None
    })

# ───── 3) Combine PSC + Liquidation → Model_Changes.xlsx ─────
all_rows = psc_rows + liq_rows
ModelChanges = pd.DataFrame(all_rows, columns=[
    'Account Change History ID',
    'Account ID',
    'Change Type',
    'Change Date',
    'Next',
    'Remove Account Change'
])
ModelChanges.to_excel('Model_Changes.xlsx', index=False)
print(f"Exported {len(ModelChanges)} account changes → Model_Changes.xlsx")

# ───── 4) Build account_number_df ─────

# 4a) Raise Cash account numbers
rc_accts = out_cash_df['Account Number'].astype(str).tolist()

# 4b) PSC / New-Model account numbers (from df_psc before we merged)
psc_accts = df_psc[RC_ACCT_COL].astype(str).tolist()

# 4c) Liquidation account numbers (from df_liq before we merged)
liq_accts = df_liq[LIQ_ACCT_COL].astype(str).tolist()

# Combine them
all_acct_numbers = rc_accts + psc_accts + liq_accts

# Create the DataFrame
account_number_df = pd.DataFrame({
    'Account Numbers': all_acct_numbers
})

# drop duplicates if you only want each number once:
account_number_df = account_number_df.drop_duplicates().reset_index(drop=True)

print(f"Collected {len(account_number_df)} account numbers → account_number_df")

c:\Users\owen.osagiede\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Generated 15 Raise Cash rows → Raise_Cash_Requests.xlsx
Exported 19 account changes → Model_Changes.xlsx
Collected 33 account numbers → account_number_df


c:\Users\owen.osagiede\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


ALERTS

In [3]:
from datetime import datetime, timedelta

# ───── after building account_number_df ─────
# Rename its column to match the notification DFs
account_number_df.rename(columns={'Account Numbers': 'Account Number'}, inplace=True)

# 5) Import Schwab Noti.csv and LPL Noti.csv 
schwab_noti_df = pd.read_csv('Files/Schwab Noti.csv')
lpl_noti_df    = pd.read_csv('Files/LPL Noti.csv')

# Filter Schwab: Subject starts with 'Move Money'
schwab_noti_df = schwab_noti_df[
    schwab_noti_df['Subject'].str.startswith('Move Money', na=False)
]

# Clean up Schwab account numbers
schwab_noti_df['Account'] = (
    schwab_noti_df['Account']
      .str.replace('-', '', regex=False)
      .astype(str)
)

# Only today’s Schwab notifications
today = datetime.today().strftime('%m/%d/%Y')
schwab_noti_df = schwab_noti_df[
    schwab_noti_df['Date Created'].str.startswith(today)
]

schwab_noti_df = schwab_noti_df[['Account']].rename(
    columns={'Account': 'Account Number'}
)
schwab_noti_df['Custodian'] = 'SCHWAB'

# Filter LPL: Category == 'Insufficient Funds'
lpl_noti_df = lpl_noti_df[
    lpl_noti_df['Category'] == 'Insufficient Funds'
]
lpl_noti_df['Account'] = (
    lpl_noti_df['Account']
      .str.replace('-', '', regex=False)
      .astype(str)
)
lpl_noti_df = lpl_noti_df[['Account']].rename(
    columns={'Account': 'Account Number'}
)
lpl_noti_df['Custodian'] = 'LPL'

# Combine notifications
noti_df = pd.concat([schwab_noti_df, lpl_noti_df], ignore_index=True)

# 1) Find overlaps
existing = set(account_number_df['Account Number'])
incoming = set(noti_df['Account Number'])
overlap = existing & incoming

if overlap:
    print(f"⚠️ {len(overlap)} account(s) already present in master list: {sorted(overlap)}")
else:
    print("✅ No notification accounts were already in the master list.")

# 2) Append only new notification accounts
new_accounts = incoming - existing
if new_accounts:
    new_df = pd.DataFrame({'Account Number': list(new_accounts)})
    account_number_df = pd.concat([account_number_df, new_df], ignore_index=True)
    account_number_df = account_number_df.drop_duplicates(subset=['Account Number'])
    print(f"Appended {len(new_accounts)} new account(s) to master list.")
else:
    print("No new accounts to append.")

✅ No notification accounts were already in the master list.
Appended 1 new account(s) to master list.


REBALANCES

In [4]:
# 1) Load the OOM.csv file
oom_df = pd.read_csv('Files/OOM.csv')

# 2) Drop any OOM rows whose Account # is in the master list
oom_df = oom_df[
    ~oom_df['Account #'].astype(int)
           .isin(account_number_df['Account Number'].astype(int))
]

# 3) Apply your existing filters
oom_df = (
    oom_df[~oom_df['Account Model']
               .str.contains('Custom|Tailored|Legacy', na=False)]
          .dropna(subset=['Account Model'])
)
oom_df = oom_df[oom_df['Account Value'] >= 100]

bad_tickers = ('RSX', 'DEWM', 'CPUT', '9999227', 'SchwabCash', 'SWGXX')
oom_df = oom_df[~oom_df['Ticker'].isin(bad_tickers)]

oom_df = (
    oom_df[~oom_df['Account Model']
               .str.contains('Liquidated', na=False)]
          .dropna(subset=['Account Model'])
)
oom_df = oom_df[oom_df['Current Units'] >= 1]

# 4) Prepare the output CSV
oom_df.rename(columns={'Account #': 'Account Number'}, inplace=True)
oom_df['Account Number'] = oom_df['Account Number'].astype(int)

rebalance_df = oom_df[['Account Number']]
rebalance_df.to_csv('Rebalance.csv', index=False)
print(f"Generated {len(rebalance_df)} rebalance rows → Rebalance.csv")

# 5) Append these new account numbers to your master list
existing = set(account_number_df['Account Number'].astype(int))
incoming = set(rebalance_df['Account Number'])
new_accounts = incoming - existing

if new_accounts:
    new_df = pd.DataFrame({'Account Number': list(new_accounts)})
    account_number_df = pd.concat([account_number_df, new_df], ignore_index=True)
    account_number_df = account_number_df.drop_duplicates(subset=['Account Number'])
    print(f"Appended {len(new_accounts)} new account(s) to master list.")
else:
    print("No new OOM accounts to append to master list.")


Generated 380 rebalance rows → Rebalance.csv
Appended 96 new account(s) to master list.


CONTRIBUTIONS

In [5]:
import pandas as pd

# 0) assume you already have:
#    account_number_df = pd.DataFrame({'Account Number': [...]}, dtype=str)

# 1) Load the Accounts.csv file (read everything as string)
accounts_df = pd.read_csv('Files/Accounts.csv', dtype=str)

# 2) Normalize Account Number → strip out non-digits, drop empties
accounts_df['Account Number'] = (
    accounts_df['Account Number']
      .fillna('')                       
      .str.replace(r'\D', '', regex=True)
)
accounts_df = accounts_df[accounts_df['Account Number'] != ""]

# 3) Drop any that are already in your master list
accounts_df = accounts_df[
    ~accounts_df['Account Number']
      .isin(account_number_df['Account Number'])
]

# 4) Filter out unwanted models
accounts_df = (
    accounts_df[~accounts_df['Model']
                         .str.contains('Custom|Liquidated|Tailored', na=False)]
               .dropna(subset=['Model'])
)

# 5) Clean up dollar columns → to float
for col in ['Current Cash $', 'Set Aside Cash Target', 'Total Value']:
    accounts_df[col] = (
        accounts_df[col]
          .str.replace('[$,]', '', regex=True)
          .astype(float)
    )

# 6) Compute True Cash Value
accounts_df['True Cash Value'] = (
    (accounts_df['Current Cash $'] - accounts_df['Set Aside Cash Target'])
    / accounts_df['Total Value']
)

# 7) Numeric filters
accounts_df = accounts_df[accounts_df['Total Value'] >= 100]
accounts_df = accounts_df[accounts_df['True Cash Value'] >= 0.03]

# 8) Export the remaining Account Numbers
contrib_df = accounts_df[['Account Number']].copy()
contrib_df.to_csv('Contributions.csv', index=False)
print(f"Exported {len(contrib_df)} accounts → Contributions.csv")

# 9) Append new ones to your master list (all as strings)
existing = set(account_number_df['Account Number'])
incoming = set(contrib_df['Account Number'])
new_accounts = incoming - existing

if new_accounts:
    new_df = pd.DataFrame({'Account Number': list(new_accounts)}, dtype=str)
    account_number_df = pd.concat([account_number_df, new_df], ignore_index=True)
    account_number_df = account_number_df.drop_duplicates(subset=['Account Number'])
    print(f"Appended {len(new_accounts)} new account(s) to master list.")
else:
    print("No new Contribution accounts to append to master list.")


Exported 3140 accounts → Contributions.csv
Appended 3140 new account(s) to master list.


In [6]:
account_number_df.to_csv('Master Account Numbers.csv', index=False)